metric_1: find a document w/same topic and value of > .3??
metric_2: find a documetn w/cosine similiarty with a topic is > .8 and value > .3
metric_3: find a document w/sum(cosine sime( > .8) and sum(val) > .3 and len(tops) <= 3

## Build up a matrix of topicsxdocuments w/value is value

In [ ]:
import pandas as pd
import gensim
import numpy
import pandas
import gensim.matutils as mt

In [ ]:
#load model
temp_file = datapath("model_LDA_mallet")
lda = Lda.load('model_LDA_mallet')
K = lda.num_topics

In [ ]:
#load training docs
cols = ['top_' + str(i) for i in K]
ind = []
lol = []
FNN = ''
filename = "corpus_bow.txt"
with open(FNN + filename, encoding='utf8') as f:
    for line in f:
        l = eval(line)
        idno = l[0]
        topics = get_document_topics(l[1])
        row = [0]*K
        for topic, value in topics:
            row[topic] = value
        ind.append(idno)
        lol.append(row)
df = pd.DataFrame(lol, columns=cols, index=ind)
df.to_csv('doc_topic_matrix.csv')

Save as CSC sparse matrix for easy access??

## Topic similarity matrix

In [ ]:
term_topic_matrix = lda.get_topics()
topic_comps = pd.DataFrame(0, index=cols, columns=cols)
indices = list(term_topic_matrix.index)
for i in range(len(indices)-1):
    for j in range(i + 1, len(indices)):
        sim = mt.cossim(term_topic_matrix.iloc[i, :], term_topic_matrix.iloc[j, :])
        topic_comps.loc[i, j] = sim
        topic_comps.loc[j, i] = sim      

## Compare topics

In [ ]:
def get_diff_topic(top, num):
    candidate_topics = list(topic_comps[topic_comps[top] > .8].sort_values(by=[top]).index)
    docs = []
    for topic in candidate_topics[:5]:
        num, doc = get_same_topic(topic)
        docs.append(doc)
    docs = sorted(docs, key=lambda d: df.loc[d, top])
    return min(num, len(docs)), docs[:num]

In [ ]:
def get_same_topic(top):
    candidate_docs = list(df[df[top] >= thresh].sort_values(by=[top]).index)
    return len(candidate_docs), candidate_docs

In [ ]:
def get_docs(top):
    num, docs = get_same_topic(top)
    num2, docs2 = get_diff_topic(top, total-num)
    return num+num2, docs.append(docs2)

## Get your top topics for document

In [ ]:
document_id = 0
topics = list(df.loc[document_id., :])
tops = [i for i in range(0,K) if topics[i] > .1]
thresh = .3
for top in tops:
    total, docs = get_docs(top, total)
    ##Do Summary w/given docs